<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%ED%9A%8C%EA%B7%80(%EB%B6%84%EB%A5%98)_%EB%AC%BC%EA%B3%A0%EA%B8%B0_%EC%96%B4%EC%A2%85_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

fish = pd.read_csv('/content/Fish.csv')
     

## Bream, Smelt
## 길이와 무게 특성만 고려

bream_smelt = fish[(fish['Species'] == 'Bream') | (fish['Species'] == 'Smelt')]

bream_smelt_train = bream_smelt[['Length1', 'Weight']]
bream_smelt_target = bream_smelt['Species']

## 학습데이터와 테스트 데이터 0.7, 0.3 비율로 만들어주세요.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

trd, tsd, trt, tst = train_test_split(bream_smelt_train, bream_smelt_target, train_size=0.7, random_state=42)

ss = StandardScaler()

scaled_trd = ss.fit_transform(trd)
scaled_tsd = ss.transform(tsd)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(scaled_trd, trt)
lr.score(scaled_trd, trt), lr.score(scaled_tsd, tst)


In [ ]:
# 시그모이드 함수 -> 입력값을 0 ~ 1 사이 값으로 환산. (확률로 사용하기 위함)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# -10부터 10까지 균일한 간격으로 100개 숫자 만들기    
x = np.linspace(-10, 10, 100)

# 시그모이드 함수 시각화
sns.lineplot(x=x, y=sigmoid(x))

In [22]:
# 로지스틱 회귀의 가중치와 편향
lr.coef_, lr.intercept_



# 결정값 (회귀식의 결과)
# y = -w1 * x - b / w2

# 결정함수
def decision_func(x) :
  return  (1.78004648 * x + 1.8820759) / -1.31525904 


# 길이가 0.31006874인 물고기의 결정값 
z = decision_func(0.31006874)

# 물고기가 양성일 확률
sigmoid(z) # 13%

# 양성 클래스(내가 관심있는 클래스), 음성 클래스(양성이 아닌 클래스)
# 두개 중 하나 분류 -> 이진 분류




z

-1.85059565847579

In [28]:
# 로지스틱 회귀 모델의 결정 함수
lr.decision_function([[ 0.31006874,  0.24553694]])

print(sigmoid(2.75695735)) # 0.05
print(lr.classes_) # 클래스 종류

lr.predict_proba([[ 0.31006874,  0.24553694]]) # 예측 확률



0.9403050743223704
['Bream' 'Smelt']


array([[0.94030507, 0.05969493]])